In [1]:
import tkinter as tk
from tkinter import *

In [2]:
import cv2


In [3]:
from PIL import Image, ImageTk
import os
import numpy as np
import cv2

In [4]:
from keras.models import Sequential

In [5]:
from keras.layers import Dense, Dropout, Flatten

In [6]:
from keras.layers import Conv2D

In [7]:
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [8]:
from keras.utils.vis_utils import plot_model
#from livelossplot import PlotLossesKerasTF
from keras.utils import plot_model

import os

In [9]:
train_dir = "C:\\Users\\ANAND\\Downloads\\archive(1)\\train"
val_dir = "C:\\Users\\ANAND\\Downloads\\archive(1)\\test"
img_size=48
batch_size=64
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_size,img_size),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')
val_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(img_size,img_size),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [10]:
for i in os.listdir("C:\\Users\\ANAND\\Downloads\\archive(1)\\train/"):
    print(str(len(os.listdir("C:\\Users\\ANAND\\Downloads\\archive(1)\\train/"+i))) +" "+ i +" images")

3995 angry images
436 disgust images
4097 fear images
7215 happy images
4965 neutral images
4830 sad images
3171 surprise images


In [11]:
for i in os.listdir("C:\\Users\\ANAND\\Downloads\\archive(1)\\test/"):
    print(str(len(os.listdir("C:\\Users\\ANAND\\Downloads\\archive(1)\\test/"+i))) +" "+ i +" images")

958 angry images
111 disgust images
1024 fear images
1774 happy images
1233 neutral images
1247 sad images
831 surprise images


In [12]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',input_shape=(48,48,1)))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())#here we get multidimention output and pass as linear to the dense so that 4*4*128=2048
emotion_model.add(Dense(1024, activation='relu'))#hddien of 1024 neurons of input 
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))#hddien of 7 neurons of input
plot_model(emotion_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)#save model leyer as model_plot.png
emotion_model.summary()

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                          

In [ ]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

C:\Users\ANAND\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
C:\Users\ANAND\AppData\Local\Temp/ipykernel_10664/4089565862.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/5
448/448 [==============================] - 2062s 5s/step - loss: 1.7984 - accuracy: 0.2611 - val_loss: 1.7125 - val_accuracy: 0.3234
Epoch 2/5
448/448 [==============================] - 2306s 5s/step - loss: 1.6352 - accuracy: 0.3633 - val_loss: 1.5529 - val_accuracy: 0.4166
Epoch 3/5
448/448 [==============================] - 2276s 5s/step - loss: 1.5283 - accuracy: 0.4119 - val_loss: 1.4635 - val_accuracy: 0.4388
Epoch 4/5
448/448 [==============================] - 731s 2s/step - loss: 1.4581 - accuracy: 0.4422 - val_loss: 1.4000 - val_accuracy: 0.4678
Epoch 5/5
303/448 [===================>..........] - ETA: 4:41 - loss: 1.4007 - accuracy: 0.4644

In [ ]:
emotion_model.save_weights('model.h5')

In [ ]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Natural", 5: "Sad", 6: "Surprised"}

#cap = cv2.VideoCapture('facial_exp.mkv')
cap = cv2.VideoCapture(0)
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    #frame = cv2.flip(frame, 1)
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(600,500),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()